In [1]:
import pandas as pd
import numpy as np

import sqlalchemy

import pandas as pd
from keys import connection_string

pd.set_option('display.max_columns', None)  # set option to display all columns
pd.options.display.max_colwidth = 1000
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
engine = sqlalchemy.create_engine(connection_string)

In [3]:
historical_data_query = """
    SELECT * FROM historical_data ORDER BY date ASC
"""
microsoft_query = """
    SELECT * FROM microsoft
"""
balance_sheet_data_query = """
    SELECT * FROM balance_sheet_data ORDER BY fiscaldateending ASC
"""
income_statement_data_query = """
    SELECT * FROM income_statement_data ORDER BY fiscaldateending ASC
"""
cash_flow_data_query = """
    SELECT * FROM cash_flow_data ORDER BY fiscaldateending ASC
"""

In [4]:
def preprocess_data(df):
    df[df.columns[0]] = pd.to_datetime(df[df.columns[0]])
    return df

In [5]:
stock_df =  pd.read_sql(historical_data_query, engine)
stock_df = preprocess_data(stock_df)
stock_df[['volume', 'split_coefficient']] = stock_df[['volume', 'split_coefficient']].astype('int64')

In [6]:
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5870 entries, 0 to 5869
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               5870 non-null   datetime64[ns]
 1   open               5870 non-null   float64       
 2   high               5870 non-null   float64       
 3   low                5870 non-null   float64       
 4   close              5870 non-null   float64       
 5   adjusted_close     5870 non-null   float64       
 6   volume             5870 non-null   int64         
 7   dividend_amount    5870 non-null   float64       
 8   split_coefficient  5870 non-null   int64         
dtypes: datetime64[ns](1), float64(6), int64(2)
memory usage: 412.9 KB


In [7]:
stock_df

,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,1999-11-01,93.25,94.19,92.12,92.37,28.87,26630600,0.00,1
1,1999-11-02,92.75,94.50,91.94,92.56,28.93,23174500,0.00,1
2,1999-11-03,92.94,93.50,91.50,92.00,28.76,22258500,0.00,1
3,1999-11-04,92.31,92.75,90.31,91.75,28.68,27119700,0.00,1
4,1999-11-05,91.81,92.87,90.50,91.56,28.62,35083700,0.00,1
...,...,...,...,...,...,...,...,...,...
5865,2023-02-23,255.56,256.84,250.48,254.77,254.77,29219095,0.00,1
5866,2023-02-24,249.96,251.00,248.10,249.22,249.22,24990905,0.00,1
5867,2023-02-27,252.46,252.82,249.39,250.16,250.16,21190042,0.00,1
5868,2023-02-28,249.07,251.49,248.73,249.42,249.42,22490950,0.00,1


In [8]:
stock_df.isnull().sum().max()

0

In [9]:
overview_df =  pd.read_sql(microsoft_query, engine)

In [10]:
overview_df.transpose()

,0
symbol,MSFT
assettype,Common Stock
name,Microsoft Corporation
description,"Microsoft Corporation is an American multinational technology company which produces computer software, consumer electronics, personal computers, and related services. Its best known software products are the Microsoft Windows line of operating systems, the Microsoft Office suite, and the Internet Explorer and Edge web browsers. Its flagship hardware products are the Xbox video game consoles and the Microsoft Surface lineup of touchscreen personal computers. Microsoft ranked No. 21 in the 2020 Fortune 500 rankings of the largest United States corporations by total revenue; it was the world's largest software maker by revenue as of 2016. It is considered one of the Big Five companies in the U.S. information technology industry, along with Google, Apple, Amazon, and Facebook."
exchange,NASDAQ
currency,USD
country,USA
sector,TECHNOLOGY
industry,SERVICES-PREPACKAGED SOFTWARE
address,"ONE MICROSOFT WAY, REDMOND, WA, US"


In [11]:
overview_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   symbol                      1 non-null      object 
 1   assettype                   1 non-null      object 
 2   name                        1 non-null      object 
 3   description                 1 non-null      object 
 4   exchange                    1 non-null      object 
 5   currency                    1 non-null      object 
 6   country                     1 non-null      object 
 7   sector                      1 non-null      object 
 8   industry                    1 non-null      object 
 9   address                     1 non-null      object 
 10  fiscalyearend               1 non-null      object 
 11  latestquarter               1 non-null      object 
 12  marketcapitalization        1 non-null      float64
 13  eps                         1 non-null 

In [12]:
balance_sheet_df =  pd.read_sql(balance_sheet_data_query, engine)
balance_sheet_df = preprocess_data(balance_sheet_df)

In [13]:
# replace 'None' with 0 in selected columns
balance_sheet_df[['treasurystock', 'deferredrevenue','capitalleaseobligations']] = balance_sheet_df[['treasurystock', 'deferredrevenue','capitalleaseobligations']].replace('None', 0)

# change datatypes to int64
int_cols = ['totalassets', 'totalcurrentassets', 'cashandcashequivalentsatcarryingvalue', 'cashandshortterminvestments',
            'inventory', 'currentnetreceivables', 'totalnoncurrentassets', 'propertyplantequipment',
            'accumulateddepreciationamortizationppe', 'intangibleassets', 'intangibleassetsexcludinggoodwill',
            'goodwill', 'investments', 'longterminvestments', 'shortterminvestments', 'othercurrentassets',
            'othernoncurrentassets', 'totalliabilities', 'totalcurrentliabilities', 'currentaccountspayable',
            'currentdebt', 'shorttermdebt', 'totalnoncurrentliabilities','longtermdebt', 'currentlongtermdebt',
            'longtermdebtnoncurrent', 'shortlongtermdebttotal', 'othercurrentliabilities', 'othernoncurrentliabilities',
            'totalshareholderequity', 'treasurystock', 'retainedearnings', 'commonstock']

balance_sheet_df[int_cols] = balance_sheet_df[int_cols].astype('int64')


balance_sheet_df[['treasurystock', 'deferredrevenue', 'shorttermdebt', 'capitalleaseobligations']] = balance_sheet_df[['treasurystock', 'deferredrevenue', 'shorttermdebt', 'capitalleaseobligations']].replace(0, np.nan)

# balance_sheet_df['treasurystock'] = balance_sheet_df['treasurystock'].replace(0, np.nan)
# balance_sheet_df['deferredrevenue'] = balance_sheet_df['deferredrevenue'].replace(0, np.nan)
# balance_sheet_df['shorttermdebt'] = balance_sheet_df['shorttermdebt'].replace(0, np.nan)
# balance_sheet_df['capitalleaseobligations'] = balance_sheet_df['capitalleaseobligations'].replace(0, np.nan)

In [14]:
balance_sheet_df

,fiscaldateending,reportedcurrency,totalassets,totalcurrentassets,cashandcashequivalentsatcarryingvalue,cashandshortterminvestments,inventory,currentnetreceivables,totalnoncurrentassets,propertyplantequipment,accumulateddepreciationamortizationppe,intangibleassets,intangibleassetsexcludinggoodwill,goodwill,investments,longterminvestments,shortterminvestments,othercurrentassets,othernoncurrentassets,totalliabilities,totalcurrentliabilities,currentaccountspayable,deferredrevenue,currentdebt,shorttermdebt,totalnoncurrentliabilities,capitalleaseobligations,longtermdebt,currentlongtermdebt,longtermdebtnoncurrent,shortlongtermdebttotal,othercurrentliabilities,othernoncurrentliabilities,totalshareholderequity,treasurystock,retainedearnings,commonstock,commonstocksharesoutstanding
0,2018-03-31,USD,245497000000,156659000000,9221000000,132270000000,2084000000,17208000000,83579000000,27929000000,28377000000,44126000000,8544000000,35582000000,135088000000,2818000000,123049000000,5097000000,7106000000,166258000000,46133000000,7623000000,23955000000,3677000000,230000000.00,114374000000,NaN,76900000000,3447000000,73480000000,77130000000,7500000000,5223000000,79239000000,NaN,9974000000,70418000000,7690000000
1,2018-06-30,USD,258848000000,169662000000,11946000000,133768000000,2662000000,26481000000,84300000000,29460000000,29223000000,43736000000,8053000000,35683000000,129579000000,1862000000,121822000000,6751000000,7442000000,176130000000,58488000000,8617000000,NaN,3998000000,NaN,112074000000,NaN,76200000000,3998000000,72242000000,76200000000,8744000000,5211000000,82718000000,NaN,13682000000,71223000000,7677000000
2,2018-09-30,USD,257619000000,164195000000,15137000000,135880000000,3614000000,17390000000,88490000000,31430000000,30953000000,43434000000,7579000000,35855000000,131941000000,2034000000,120743000000,7311000000,9792000000,171652000000,56277000000,8511000000,NaN,6497000000,NaN,109723000000,NaN,76200000000,6497000000,69733000000,76200000000,7953000000,5539000000,85967000000,NaN,17279000000,71303000000,7680000000
3,2018-12-31,USD,258859000000,156874000000,6638000000,127662000000,1961000000,19680000000,97079000000,32717000000,33082000000,50059000000,8482000000,41577000000,124407000000,2274000000,121024000000,7571000000,10129000000,166731000000,50318000000,7563000000,NaN,3516000000,NaN,110730000000,NaN,73200000000,3516000000,69653000000,73200000000,8297000000,6055000000,92128000000,NaN,16585000000,77556000000,7683000000
4,2019-03-31,USD,263281000000,159887000000,11212000000,131618000000,1951000000,19269000000,98273000000,33648000000,35431000000,49964000000,8103000000,41861000000,127455000000,2403000000,120406000000,7049000000,10258000000,168417000000,53861000000,7544000000,NaN,6515000000,NaN,108584000000,5972000000,73100000000,6515000000,66585000000,73100000000,7837000000,6763000000,94864000000,NaN,18338000000,77791000000,7666000000
5,2019-06-30,USD,286556000000,175552000000,11356000000,133819000000,2063000000,29524000000,105825000000,36477000000,35330000000,49776000000,7750000000,42026000000,129789000000,2649000000,122463000000,10146000000,14723000000,184226000000,69420000000,9382000000,74412000000,5516000000,NaN,108618000000,6188000000,72178000000,5516000000,66662000000,72200000000,9351000000,7581000000,102330000000,NaN,24150000000,78520000000,7643000000
6,2019-09-30,USD,278955000000,165896000000,13117000000,136636000000,2622000000,19087000000,115359000000,38409000000,36971000000,49621000000,7508000000,42113000000,131971000000,2684000000,123519000000,7551000000,14455000000,172894000000,58118000000,8574000000,68052000000,3017000000,3017000000.00,114776000000,6659000000,69495000000,3017000000,66478000000,69495000000,8507000000,8826000000,106061000000,NaN,27240000000,78882000000,7634000000
7,2019-12-31,USD,282794000000,167074000000,8864000000,134253000000,1823000000,23525000000,118120000000,40522000000,39597000000,49374000000,7126000000,42248000000,129483000000,2755000000,125389000000,7473000000,14630000000,172685000000,59640000000,88110000

In [15]:
balance_sheet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   fiscaldateending                        20 non-null     datetime64[ns]
 1   reportedcurrency                        20 non-null     object        
 2   totalassets                             20 non-null     int64         
 3   totalcurrentassets                      20 non-null     int64         
 4   cashandcashequivalentsatcarryingvalue   20 non-null     int64         
 5   cashandshortterminvestments             20 non-null     int64         
 6   inventory                               20 non-null     int64         
 7   currentnetreceivables                   20 non-null     int64         
 8   totalnoncurrentassets                   20 non-null     int64         
 9   propertyplantequipment                  20 non-null     

In [16]:
# sads

NameError: name 'sads' is not defined

In [17]:
income_statement_df =  pd.read_sql(income_statement_data_query, engine)
income_statement_df = preprocess_data(income_statement_df)

In [18]:
income_statement_df

,fiscaldateending,reportedcurrency,grossprofit,totalrevenue,costofrevenue,costofgoodsandservicessold,operatingincome,sellinggeneralandadministrative,researchanddevelopment,operatingexpenses,investmentincomenet,netinterestincome,interestincome,interestexpense,noninterestincome,othernonoperatingincome,depreciation,depreciationandamortization,incomebeforetax,incometaxexpense,interestanddebtexpense,netincomefromcontinuingoperations,comprehensiveincomenetoftax,ebit,ebitda,netincome
0,2018-03-31,USD,17550000000.00,26819000000.00,9269000000.00,9269000000.00,8292000000.00,5543000000.00,3715000000.00,9818000000.00,575000000.00,-691000000.00,None,691000000.00,-52000000.00,7000000.00,None,560000000.00,8641000000.00,1217000000.00,691000000.00,7424000000.00,6670000000.00,9332000000.00,9892000000.00,7424000000.00
1,2018-06-30,USD,20343000000.00,30085000000.00,34165000000.00,9742000000.00,10379000000.00,6031000000.00,3933000000.00,10464000000.00,636000000.00,-672000000.00,None,672000000.00,30085000000.00,-61000000.00,None,500000000.00,10680000000.00,1807000000.00,672000000.00,10379000000.00,7797000000.00,11352000000.00,11852000000.00,8873000000.00
2,2018-09-30,USD,19179000000.00,28615000000.00,15708000000.00,9905000000.00,9955000000.00,5247000000.00,3977000000.00,9780000000.00,681000000.00,-674000000.00,None,674000000.00,29289000000.00,14000000.00,None,556000000.00,10221000000.00,1397000000.00,674000000.00,8824000000.00,8463000000.00,10895000000.00,11451000000.00,8824000000.00
3,2018-12-31,USD,20048000000.00,31796000000.00,18673000000.00,12423000000.00,10258000000.00,5720000000.00,4070000000.00,10320000000.00,704000000.00,-672000000.00,None,672000000.00,32468000000.00,34000000.00,None,530000000.00,10385000000.00,1965000000.00,672000000.00,8420000000.00,9022000000.00,11057000000.00,11587000000.00,8420000000.00
4,2019-03-31,USD,20401000000.00,29993000000.00,16345000000.00,10170000000.00,10341000000.00,5744000000.00,4316000000.00,10491000000.00,668000000.00,-671000000.00,None,671000000.00,30664000000.00,16000000.00,None,431000000.00,10486000000.00,1677000000.00,671000000.00,8809000000.00,9557000000.00,11157000000.00,11588000000.00,8809000000.00
5,2019-06-30,USD,23305000000.00,33717000000.00,17199000000.00,10412000000.00,12405000000.00,6387000000.00,4513000000.00,11300000000.00,709000000.00,-669000000.00,321000000,669000000.00,33717000000.00,-121000000.00,None,400000000.00,12596000000.00,-591000000.00,669000000.00,12405000000.00,14112000000.00,13265000000.00,13665000000.00,13187000000.00
6,2019-09-30,USD,22649000000.00,32428000000.00,16363000000.00,10406000000.00,12686000000.00,5398000000.00,4565000000.00,10522000000.00,724000000.00,-637000000.00,139000000,637000000.00,33065000000.00,-18000000.00,None,559000000.00,12686000000.00,2008000000.00,637000000.00,10678000000.00,10957000000.00,13323000000.00,13882000000.00,10678000000.00
7,2019-12-31,USD,24548000000.00,36322000000.00,18986000000.00,12358000000.00,13891000000.00,6054000000.00,4603000000.00,11231000000.00,688000000.00,-654000000.00,281000000,654000000.00,36976000000.00,-19000000.00,None,574000000.00,14085000000.00,2436000000.00,654000000.00,11649000000.00,11455000000.00,14739000000.00,15313000000.00,11649000000.00
8,2020-03-31,USD,24046000000.00,34315000000.00,17650000000.00,10975000000.00,12975000000.00,6184000000.00,4887000000.00,11562000000.00,673000000.00,-614000000.00,153000000,614000000.00,34929000000.00,-3000000.00,None,491000000.00,12843000000.00,2091000000.00,614000000.00,10752000000.00,13683000000.00,13457000000.00,13948000000.00,10752000000.00
9,2020-06-30,USD,25694000000.00,38033000000.00,20161000000.00,12339000000.00,13407000000.00,7073000000.00,5214000000.00,13036000000.00,595000000.00,-686000000.00,None,686000000.00,38033000000.00,0.00,None,588000000.00,13422000000.00,2220000000.00,686000000.00,13407000000.00,11712000000.00,14108000000.00,14696000000.00,11202000000.00


In [19]:
income_statement_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   fiscaldateending                   20 non-null     datetime64[ns]
 1   reportedcurrency                   20 non-null     object        
 2   grossprofit                        20 non-null     float64       
 3   totalrevenue                       20 non-null     float64       
 4   costofrevenue                      20 non-null     float64       
 5   costofgoodsandservicessold         20 non-null     float64       
 6   operatingincome                    20 non-null     float64       
 7   sellinggeneralandadministrative    20 non-null     float64       
 8   researchanddevelopment             20 non-null     float64       
 9   operatingexpenses                  20 non-null     float64       
 10  investmentincomenet                20 no

In [ ]:
cash_flow_df =  pd.read_sql(cash_flow_data_query, engine)
cash_flow_df = preprocess_data(cash_flow_df)

In [ ]:
cash_flow_df

In [ ]:
cash_flow_df.info()